In [ ]:
%matplotlib inline

import datacube
from datacube.testutils import io


In [ ]:
dc = datacube.Datacube(app='s2')
dc.list_products()

In [ ]:
query = {}
query['product'] = 'ls8_usgs_sr_scene'
query['id'] = '13b9c5f0-65f0-5d2e-8473-5bbfc92a9673'
query['measurements'] = ['green','swir1']

In [ ]:
ds = dc.find_datasets(**query)

In [ ]:
query['output_crs'] = ds[0].metadata_doc['grid_spatial']['projection']['spatial_reference']
query['resolution'] = (-30,30)

In [ ]:
l2_data = dc.load(**query)

In [ ]:
l2_data

In [ ]:
l1_green = io.rio_slurp_xarray('LC08_L1TP_204052_20180109_20180119_01_T1_B3.tiff')

In [ ]:
l1_swir1 =  io.rio_slurp_xarray('LC08_L1TP_204052_20180109_20180119_01_T1_B6.tiff')

In [ ]:
l1_green

## From here it's Ee Faye

### Calculating MNDWI for Level 2 data

In [ ]:
# This def is from Caitlin's notebook, note NVDI and NDWI have been removed here
# Note typo in original def - MNDWI incorrectly uses NIR in the denominator

def calculate_band_indices(dc):
    dc['mndwi'] = (dc.green - dc.swir1)/(dc.green + dc.swir1)
    return(dc)

dc_level2 = calculate_band_indices(l2_data)

In [ ]:
print(dc_level2.time)

# There is only one timestep here. I'm not sure why. It does not correspond to Caitlin's timesteps

### Calculating MNDWI for Level 1 data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dc_level1 = np.nan_to_num((l1_green - l1_swir1)/(l1_green + l1_swir1))
print(dc_level1)

In [ ]:
print(l1_green)

In [ ]:
print(l1_swir1 / l1_green)

In [ ]:
#divtest = np.divide(l1_swir1, l1_green, out=np.zeros_like(l1_swir1), where=l1_green!=0)
divtest2 = np.nan_to_num(l1_swir1/l1_green)
print(divtest2)

In [ ]:
l1_green.plot(figsize = (10,10))

In [ ]:
#divtest2.plot(figsize = (10,10))
divtest2.shape
#divtest2.dtype

In [ ]:
l1_green.shape
l1_green.dtype

In [ ]:
divtest2.shape

In [ ]:
np.isnan(divtest2).any()

In [ ]:
divtest2_int=divtest2.astype(np.uint16)

In [ ]:
plt.plot(divtest2_int)
plt.show()

In [ ]:
#divtest2_int.plot(figsize=(10,10))
plt.imshow(divtest2_int)
plt.show()

## Now that plotting works - to calculate the actual MNDWI

In [ ]:
dc_level2

In [ ]:
dc_level2['mndwi'].plot(figsize=(10,10))

In [ ]:
mndwi_level1 = np.nan_to_num((l1_green - l1_swir1)/(l1_green + l1_swir1))

In [ ]:
plt.imshow(mndwi_level1)
plt.show()

## Just kidding, let's see if we can convert the Level 1 data to an xarray Dataset

In [ ]:
import pandas as pd
import xarray as xr

In [ ]:
l1xarraytest = xr.DataArray(mndwi_level1, dims=['y','x'])
l1xarraytest

In [ ]:
np.isnan(mndwi_level1).any()

np.count_nonzero(mndwi_level1)

In [ ]:
l1xarraytest.plot(figsize = (10,10), robust = True)

In [ ]:
np.max(mndwi_level1)

## Okay something is not right with the index calculation

mndwi_level1 = np.nan_to_num((l1_green - l1_swir1)/(l1_green + l1_swir1))

In [ ]:
np.max(l1_swir1)

In [ ]:
np.max(l1_green)

In [ ]:
numerator = l1_green - l1_swir1
numerator

In [ ]:
np.max(numerator)

In [ ]:
denominator = l1_green + l1_swir1
denominator

In [ ]:
mndwi = numerator/denominator
mndwi

In [ ]:
np.max(mndwi)

In [ ]:
np.max(l1_green)

In [ ]:
np.max(l1_swir1)

In [ ]:
ds = l1_green.to_dataset(name = 'green')

In [ ]:
ds['swir1'] = l1_swir1

In [ ]:
ds

In [ ]:
import sys
#sys.path.append("../Scripts")
#from deafrica_bandindices import calculate_indices

In [ ]:
sys.path.append("../Scripts")
from deafrica_bandindices import calculate_indices

In [ ]:
ds = calculate_indices(ds, index = 'MNDWI', collection = 'c1') # use drop = True to get rid of bands

In [ ]:
ds.MNDWI.plot(figsize=(10,10))